### Board Game Anlysis

I have created this Board game analysis for two reasons
* I love board games and would like to explore this data
* I would like to show what I know!

I pulled this dataset off Kaggle. Orginally the data was pulled from boardgamegeek.

In [1]:
#Imports
import numpy as np
import pandas as pd
import csv
import matplotlib
import re
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error


In [2]:
#Read in CSV file
Games=pd.read_csv("bgg_dataset.csv", sep=";")
#First 5 rows
Games.head(5)

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,"8,61",2,"2,84",65294.0,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,70",5,"4,22",16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


In [3]:
#How much data do we have?
print(len(Games), "rows in the dataset.")

20343 rows in the dataset.


In [4]:
#Look if there is missing data
Games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  20327 non-null  float64
 1   Name                20343 non-null  object 
 2   Year Published      20342 non-null  float64
 3   Min Players         20343 non-null  int64  
 4   Max Players         20343 non-null  int64  
 5   Play Time           20343 non-null  int64  
 6   Min Age             20343 non-null  int64  
 7   Users Rated         20343 non-null  int64  
 8   Rating Average      20343 non-null  object 
 9   BGG Rank            20343 non-null  int64  
 10  Complexity Average  20343 non-null  object 
 11  Owned Users         20320 non-null  float64
 12  Mechanics           18745 non-null  object 
 13  Domains             10184 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 2.2+ MB


It looks like we may have some missing values in our ID. This could be expansions to games. Rating and complexity averages are not rerestiring as numbers. If you look back at the head you can see it is comma seperated so this will need to be fixed. I am also going to drop the owned users data thats missing. These would be games no one has access to yet. These maybe fun to test towards the end of our analysis to see if we can predict their ratings based off mechanics or other vairables.

### Data Cleaning

In [5]:
# Create a function to fix comma data for Rating and Complexity
def commaremoval(data):
    Newlist=[]
    for i in data:
        i = i.replace(",",".")
        Newlist.append(i)
    return Newlist
Games["Rating Average"] = pd.to_numeric(commaremoval(Games["Rating Average"]))
Games["Complexity Average"] = pd.to_numeric(commaremoval(Games["Complexity Average"]))

In [6]:
#Drop Games with no owners
Games=Games[Games["Owned Users"].notna()]

In [7]:
#Change all / to , for easier seperation
Games["Mechanics"] = Games["Mechanics"].str.replace("/",",")
Games["Domains"] = Games["Domains"].str.replace("/",",")

In [8]:
#Checking Data Again
Games.head(5)

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,3.86,68323.0,"Action Queue, Action Retrieval, Campaign , Bat...","Strategy Games, Thematic Games"
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,8.61,2,2.84,65294.0,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,3.91,28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,3.24,87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,4.22,16831.0,"Action Drafting, Area Majority , Influence, Ar...","Strategy Games, Thematic Games"


We now have a nice list of 20320 Games to look at! Lets continue to break down the Mechanics and Domains in to usable data for Scikit. I will use One Hot Encoding.

In [9]:
#One Hot Encode Mechanics
GamesOHEM = Games['Mechanics'].str.get_dummies(sep=',')
#df['string_column'].str.get_dummies(sep = ',')
#One Hot Encode Domains
GamesOHED=Games['Domains'].str.get_dummies(sep=',')
#Join tables
Games1 = pd.concat([GamesOHEM, GamesOHED], axis=1)
GamesF=pd.concat([Games, GamesOHEM], axis=1)
GamesF.head()

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,...,Team-Based Game,Tile Placement,Time Track,Trading,Traitor Game,Trick-taking,Variable Phase Order,Variable Player Powers,Voting,Worker Placement
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,...,0,0,0,0,0,0,0,0,0,0
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,8.61,2,...,0,0,0,0,0,0,0,0,0,0
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,...,0,0,0,0,0,0,0,0,0,0
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,...,0,0,0,0,0,0,0,0,0,0
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,...,0,0,0,0,0,0,0,0,0,0


### Data Analyzation
Questions I want to Answer

* Averages for players, play time,etc. 
* What Mechanics seem to be the most popular?
* What Mechanics seem to correlate to the highest scores?
* Can we create a model to predict success of a game?

In [10]:
#Averages
Games.describe()

,ID,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users
count,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000,20320.000000
mean,108210.740748,1984.226230,2.019636,5.673327,91.326772,9.600246,841.778691,6.403363,10170.563976,1.990994,1408.457628
std,98678.347583,214.117399,0.690545,15.239657,545.749554,3.645790,3513.464339,0.935762,5873.389392,0.849022,5040.179315
min,1.000000,-3500.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.050000,1.000000,0.000000,0.000000
25%,11035.250000,2001.000000,2.000000,4.000000,30.000000,8.000000,55.000000,5.820000,5084.750000,1.330000,146.000000
50%,88928.000000,2011.000000,2.000000,4.000000,45.000000,10.000000,120.000000,6.430000,10168.500000,1.970000,309.000000
75%,192924.750000,2016.000000,2.000000,6.000000,90.000000,12.000000,385.000000,7.030000,15258.250000,2.540000,864.000000
max,331787.000000,2022.000000,10.000000,999.000000,60000.000000,25.000000,102214.000000,9.580000,20344.000000,5.000000,155312.000000


There is a game that plays for 600000 minutes? Thats 40 days lets see.

In [11]:
Games.nlargest(5,['Play Time'])

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
13420,4815.0,The Campaign for North Africa: The Desert War ...,1979.0,8,10,60000,14,146,6.10,13422,4.71,385.0,"Dice Rolling, Hexagon Grid, Simulation",Wargames
3208,29285.0,Case Blue,2007.0,2,2,22500,12,289,8.26,3210,4.58,711.0,"Dice Rolling, Hexagon Grid, Simulation",Wargames
6035,46669.0,1914: Offensive à outrance,2013.0,2,4,17280,0,108,7.98,6037,4.07,661.0,"Dice Rolling, Hexagon Grid, Simulation",Wargames
7895,158793.0,Atlantic Wall: D-Day to Falaise,2014.0,2,6,14400,16,69,8.08,7897,4.89,328.0,"Hexagon Grid, Zone of Control",Wargames
1322,254.0,Empires in Arms,1983.0,2,7,12000,14,1238,7.60,1323,4.42,2129.0,"Area Movement, Dice Rolling, Movement Points, ...",Wargames


In [12]:
#Mechanics Popular and High Scoring

### Model Creation

In [13]:
#Create Feature Variables
a=GamesF.drop(['Mechanics','Domains','Name','ID'],axis=1)
b=a
scaler = StandardScaler()
scaler.fit(a)
X=a.drop(['Rating Average'],axis=1)
y=b['Rating Average']




In [14]:
#Split Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101)
print('Train size: ', len(X_train), 'Test size: ', len(X_test))

Train size:  16256 Test size:  4064


In [15]:
#multiple Linear Regression
lr_model=LinearRegression()
lr_model.fit(X_train,y_train)

y_pred = lr_model.predict(X)
print('Results for multiple linear regression on training data')
print(' Default settings')
print('Internal parameters:')
print(' Bias is ', lr_model.intercept_)
print(' Coefficients', lr_model.coef_)
print(' Score', lr_model.score(X,y))
print('MAE is ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2 ', r2_score(y,y_pred))


Results for multiple linear regression on training data
 Default settings
Internal parameters:
 Bias is  7.043382972363941
 Coefficients [ 6.45113636e-05 -5.14607679e-02 -9.78057167e-05  8.01734172e-05
 -7.06958194e-03  7.55559886e-05 -1.02447853e-04  1.66610142e-01
 -6.59468875e-05  4.92524578e+06 -1.25478634e+00  9.41263972e-02
  1.04212028e-01 -2.82753574e-01  1.26011142e-01  3.20396694e-01
 -1.90231635e+06 -2.15816655e-02  1.51096439e-01 -2.64250526e+06
 -5.02572234e-01  3.33718782e-01 -2.68117154e-01  1.42099983e+06
  1.30727154e-01 -1.53935675e-01 -2.87045753e-02  7.65136299e-02
 -1.73293527e-01 -4.42161428e+06  9.43869844e-02 -1.53134763e-01
 -1.47466276e+00  1.57201152e-01  4.42161440e+06  4.29348259e+05
 -1.68811039e-02  3.65640774e-01 -2.24574839e-01 -1.49203708e-01
  3.28088083e-01  2.90133496e-01  1.13563337e+06  2.38985631e-01
 -1.84366169e-01  1.43906507e-02 -2.11040180e-01 -1.31998535e-01
  2.85859757e-01  2.13125049e-01 -2.33940105e-02  3.39229955e-01
  6.36724576e-02  

That was ugly. The R Squared value is very very negative meaning we would be better off just averaging the values lets try another approach.

In [16]:
# Decision Tree Regressor
DTR_model=DecisionTreeRegressor()
DTR_model.fit(X_train,y_train)

y_pred = DTR_model.predict(X)
print('Results for Decsion Tree Regressor on training data')
print(' Default settings')
print(' Score', DTR_model.score(X,y))
print('MAE is ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2 ', r2_score(y,y_pred))



Results for Decsion Tree Regressor on training data
 Default settings
 Score 0.9766182278757504
MAE is  0.038441929133858264
RMSE is  0.14308472381416676
MSE is  0.020473238188976377
R^2  0.9766182278757504


That is much better. Looking at just the training set we have a high correlation between the model and the data points. Lets try it with the test data

In [17]:
y_pred_test = DTR_model.predict(X_test)
print('Results for Decsion Tree Regressor on test data')
print(' Default settings')
print(' Score', DTR_model.score(X_test,y_test))
print('MAE is ', mean_absolute_error(y_test, y_pred_test))
print('RMSE is ', np.sqrt(mean_squared_error(y_test, y_pred_test)))
print('MSE is ', mean_squared_error(y_test, y_pred_test))
print('R^2 ', r2_score(y_test,y_pred_test))

Results for Decsion Tree Regressor on test data
 Default settings
 Score 0.8834015195949152
MAE is  0.19220964566929133
RMSE is  0.3199471689902598
MSE is  0.10236619094488189
R^2  0.8834015195949152


It looks like the model maybe over fitting. Our R squared value dropped by about 10%. .88 is still a pretty good score but lets try a couple more models.

In [20]:
# Random Forrest Regressor
from sklearn.ensemble import RandomForestRegressor

RFR_model=RandomForestRegressor()
RFR_model.fit(X_train,y_train)

y_pred = RFR_model.predict(X)
print('Results for Decsion Tree Regressor on training data')
print(' Default settings')
print(' Score', RFR_model.score(X,y))
print('MAE is ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2 ', r2_score(y,y_pred))

y_pred_test = RFR_model.predict(X_test)
print('Results for Decsion Tree Regressor on training data')
print(' Default settings')
print(' Score', RFR_model.score(X_test,y_test))
print('MAE is ', mean_absolute_error(y_test, y_pred_test))
print('RMSE is ', np.sqrt(mean_squared_error(y_test, y_pred_test)))
print('MSE is ', mean_squared_error(y_test, y_pred_test))
print('R^2 ', r2_score(y_test,y_pred_test))

Results for Decsion Tree Regressor on training data
 Default settings
 Score 0.9808781789093595
MAE is  0.06550221948818903
RMSE is  0.12939550294953456
MSE is  0.016743196183563007
R^2  0.9808781789093595
Results for Decsion Tree Regressor on training data
 Default settings
 Score 0.93819937667424
MAE is  0.13128944389763783
RMSE is  0.23293151653474722
MSE is  0.05425709139517722
R^2  0.93819937667424


This is pretty good lets try a Neural Network Regressor just to see.

In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

NN_model=MLPRegressor()
NN_model.fit(X_train,y_train)

y_pred = NN_model.predict(X)
print('Results for Neural Net Regressor on training data')
print(' Default settings')
print(' Score', NN_model.score(X,y))
print('MAE is ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2 ', r2_score(y,y_pred))

y_pred_test = NN_model.predict(X_test)
print('Results for Neural Net Regressor on training data')
print(' Default settings')
print(' Score', NN_model.score(X_test,y_test))
print('MAE is ', mean_absolute_error(y_test, y_pred_test))
print('RMSE is ', np.sqrt(mean_squared_error(y_test, y_pred_test)))
print('MSE is ', mean_squared_error(y_test, y_pred_test))
print('R^2 ', r2_score(y_test,y_pred_test))

Results for Neural Net Regressor on training data
 Default settings
 Score -116.40001153058222
MAE is  3.3884343016033713
RMSE is  10.138848357227657
MSE is  102.79624601085796
R^2  -116.40001153058222
Results for Neural Net Regressor on training data
 Default settings
 Score -165.07829812322692
MAE is  3.632913474412807
RMSE is  12.075031417778217
MSE is  145.806383740331
R^2  -165.07829812322692


The random forrest Regressor seems to work the best for predicting our board game rating. We could set up a pipeline that allows you to plug your board games criteria in through an API and it will predict the rating of your game.